In [1]:
import praw
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import pandas as pd
import datetime

import secrets

In [2]:
HF_TOKENIZER_NEWS_CLASSIFIER = "mrm8488/bert-mini-finetuned-age_news-classification"
HF_MODEL_NEWS_CLASSIFIER = "mrm8488/bert-mini-finetuned-age_news-classification"

SUBREDDITS = "wallstreetbets+investing+options+pennystocks+options+stocks"

NUM_SUBMISSION_TO_GET = 100

NEWS_CLASSES = ["world", "sports", "business", "sci/tech"]

In [3]:
news_tokenizer = AutoTokenizer.from_pretrained(HF_TOKENIZER_NEWS_CLASSIFIER)
news_model = AutoModelForSequenceClassification.from_pretrained(HF_MODEL_NEWS_CLASSIFIER)
sentiment_model = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [4]:
reddit = praw.Reddit(
    client_id=secrets.REDDIT_API_CLIENT_ID,
    client_secret=secrets.REDDIT_API_CLIENT_SECRET,
    user_agent=secrets.REDDIT_API_USER_AGENT
)
subreddit = reddit.subreddit(SUBREDDITS)

In [5]:
subreddit

Subreddit(display_name='wallstreetbets+investing+options+pennystocks+options+stocks')

In [6]:
submissions = []

for submission in subreddit.stream.submissions():
    submissions.append(submission)
    if len(submissions) > NUM_SUBMISSION_TO_GET:
        break

KeyboardInterrupt: 

In [50]:
print(subreddit)
top_submissions = []
for submission in subreddit.hot(limit=10):
    top_submissions.append(submission)
    if len(submissions) > NUM_SUBMISSION_TO_GET:
        break
len(top_submissions)

wallstreetbets+investing+options+pennystocks+options+stocks


1

In [7]:
top_submissions = []
for submission in reddit.subreddit("wallstreetbets").top(limit=1000):
    top_submissions.append(submission)

print(len(top_submissions))

975


In [8]:
from praw.models import MoreComments

NUM_COMMENTS = 10000
comments = 0
conversations = []

for submission in top_submissions:
    for top_level_comment in submission.comments:
        convo = []
        if isinstance(top_level_comment, MoreComments):
            continue
        convo.append(top_level_comment.body)
        for reply in top_level_comment.replies:
            if isinstance(reply, MoreComments):
                continue
            convo.append(reply.body)
        
        conversations.append(convo)
        
        comments = comments + 1
        print(comments)
    if comments > NUM_COMMENTS:
        break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [10]:
conversations

[['Someone needs to open a GameStop across the ny stock exchange 💎🙌',
  'We can do it. We own GameStop 💎🙌'],
 ['Bro wtf... 2021 is crazy.',
  'When the going gets weird, the weird turn pro. - Hunter S Thompson',
  'The writers of 2021 are nuts, really going hard after 2020',
  'Loving this year already',
  '2021 is chaotic neutral\n\n2020 was true evil',
  'We’ve all been hardened by the pandemic',
  "2020 was an appetizer... Hopefully the shit will roll UP-hill this year... It's on us. I think this whole thing that WSB started is showing everyone what we can get accomplished if we can work together"],
 ['This is the hype we need for next week!\n\nWE LIKE THE STOCK!',
  '[WE LIKE THE STOCK](https://www.youtube.com/watch?v=71JOIJKbse0&ab_channel=placeboing)'],
 ['This is easily one of the greatest things to happen during our lives.',
  'WE REALLY LIKE THE STOCK!',
  '[deleted]',
  "The market is there to serve the people, all of the people. \n\nIf that's not the case, it is no longer co

In [12]:
preprocessed_conversations = []

for convo in conversations:
    the_convo = []
    for comment in convo:
        c = comment.lower().replace(".", " .").replace("?", " ?").replace("!", " !")
        the_convo.append(c)
    preprocessed_conversations.append(the_convo)

In [14]:
flat_convos = []

for convo in preprocessed_conversations:
    for comment in convo:
        flat_convos.append(comment)

In [33]:
import random

def create_random_candidates():
    candidates = []
    for _ in range(3):
        candidates.append(random.choice(flat_convos))
    return candidates

In [59]:
import copy

personality = [
    "i like stocks",
    "i am from wallstreetbets",
    "i like reddit",
    "i love holding the stock"
]

training_data = []

for convo in preprocessed_conversations:
    the_convo = {}
    utterances = []
    convo_len = len(convo)
    history = []
    total_words = 0
    if convo_len > 1:
        for i, comment in enumerate(convo):
            convo_obj = {}
            if i % 2 == 0:
                if convo_len >= i+2:
                    len_words = len(comment) + len(convo[i+1])
                    total_words = total_words + len_words
                    if total_words > 400:
                        continue
                    else:
                        history.append(comment)
                        convo_obj["candidates"] = create_random_candidates()
                        convo_obj["candidates"].append(convo[i+1])
                        convo_obj["history"] = copy.deepcopy(history)
                        utterances.append(convo_obj)
                        total_words = total_words + len(convo_obj["candidates"][0]) + len(convo_obj["candidates"][1]) + len(convo_obj["candidates"][2])
                        if len(utterances) > 0 and total_words < 400:
                            the_convo["personality"] = personality
                            the_convo["utterances"] = utterances
                            training_data.append(the_convo)

training_data

[{'personality': ['i like stocks',
   'i am from wallstreetbets',
   'i like reddit',
   'i love holding the stock'],
  'utterances': [{'candidates': ['great meme',
     'ape together strong !',
     '[deleted]',
     'we can do it . we own gamestop 💎🙌'],
    'history': ['someone needs to open a gamestop across the ny stock exchange 💎🙌']}]},
 {'personality': ['i like stocks',
   'i am from wallstreetbets',
   'i like reddit',
   'i love holding the stock'],
  'utterances': [{'candidates': ['gme hasn’t even reached its peak yet and apes have already done more good than melvin has ever done',
     'i think we all needed this after today . a sincere thank you u/deepfuckingvalue from all your fellow apes 💎🙌🦍',
     'any coincidence that there was a full moon the other day ? 🚀🌕',
     'when the going gets weird, the weird turn pro . - hunter s thompson'],
    'history': ['bro wtf . . . 2021 is crazy .']}]},
 {'personality': ['i like stocks',
   'i am from wallstreetbets',
   'i like reddit'

In [60]:
import json

with open("minimal_train.json", "w") as f:
    json.dump(training_data, f)


In [3]:
model.train_model("data/minimal_train.json")

100%|██████████| 959/959 [00:07<00:00, 125.76it/s]
C:\Users\markr\anaconda3\envs\reddit-broker-bot\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1 of 1:   0%|          | 0/1 [00:03<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 100.00 MiB (GPU 0; 8.00 GiB total capacity; 6.67 GiB already allocated; 0 bytes free; 6.90 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [46]:
top_submissions

[Submission(id='l8rf4k')]

In [8]:
submissions

[Submission(id='tspf5n'),
 Submission(id='tspj31'),
 Submission(id='tspjs8'),
 Submission(id='tsplmo'),
 Submission(id='tspp2e'),
 Submission(id='tsps2i'),
 Submission(id='tspsb1'),
 Submission(id='tspuz4'),
 Submission(id='tspw5a'),
 Submission(id='tspxm2'),
 Submission(id='tspyte'),
 Submission(id='tspyvx'),
 Submission(id='tsq031'),
 Submission(id='tsq03c'),
 Submission(id='tsqtaj'),
 Submission(id='tsqvlv'),
 Submission(id='tsqzuf'),
 Submission(id='tsqzwg'),
 Submission(id='tsra3w'),
 Submission(id='tsrt2v'),
 Submission(id='tsrwcp'),
 Submission(id='tssmea'),
 Submission(id='tssnhp'),
 Submission(id='tssp43'),
 Submission(id='tst6or'),
 Submission(id='tst9gq'),
 Submission(id='tstakp'),
 Submission(id='tsusyz'),
 Submission(id='tsvb08'),
 Submission(id='tsvb8s'),
 Submission(id='tsvh5h'),
 Submission(id='tsvlwn'),
 Submission(id='tsvqr5'),
 Submission(id='tsvqzf'),
 Submission(id='tsvxos'),
 Submission(id='tsvzlv'),
 Submission(id='tsw2li'),
 Submission(id='tsw8of'),
 Submission(

In [11]:
class RedditSubmission():
    subreddit: str
    title: str
    time_created: datetime.datetime
    author: str
    inference_subject: str
    inference_sentiment: str

    def __init__(self, subreddit: str, title: str, time_created: str, author: str, inference_subject: str = None, inference_sentiment: str = None):
        self.subreddit = subreddit
        self.title = title
        self.time_created = self.convert_time_to_datetime(time_created)
        self.author = author
        self.inference_subject = self.run_subject_analysis()
        self.inference_sentiment = self.run_sentiment_analysis()

    def convert_time_to_datetime(self, time_created) -> datetime.datetime:
        dt = datetime.datetime.fromtimestamp(time_created)
        return dt

    def map_news_output_to_class(self, inference_output: torch.Tensor) -> str:
        softmax_values = []
        for output in inference_output:
            softmax_values.append(output.item())
        max_value = max(softmax_values)
        max_index = softmax_values.index(max_value)
        return NEWS_CLASSES[max_index]
    
    def run_subject_analysis(self) -> str:
        inputs = news_tokenizer(self.title, return_tensors="pt")
        labels = torch.tensor([1]).unsqueeze(0) # Batch size of 1
        outputs = news_model(**inputs, labels=labels) # Unpack key-value pairs into keyword args in function call
        news_subject = self.map_news_output_to_class(outputs.logits[0]) # Taking softmax tensor from inference
        return news_subject

    def run_sentiment_analysis(self) -> str:
        sentiment = sentiment_model(self.title)
        return sentiment[0]["label"]

In [12]:
reddit_submission_objects = []

for submission in submissions:
    s = RedditSubmission(submission.subreddit, submission.title, submission.created_utc, submission.author)
    reddit_submission_objects.append(s)

In [29]:
df = pd.DataFrame([vars(submission) for submission in reddit_submission_objects])
list_of_titles = list(df["title"])
df

,subreddit,title,time_created,author,inference_subject,inference_sentiment
0,wallstreetbets,Short it,2022-03-30 21:16:41,VarianceOvertime,world,NEGATIVE
1,wallstreetbets,President Biden expected to announce massive r...,2022-03-30 21:22:33,northdancer,business,POSITIVE
2,wallstreetbets,lunch is for whimps- I got a connection @THEE ...,2022-03-30 21:23:43,Redskin_Flippy,business,POSITIVE
3,pennystocks,Can you remind me why I bought CTHR?,2022-03-30 21:26:31,watching_waiting_0,business,NEGATIVE
4,options,OTM TECH CALLS 3ISH MONTH FROM EXPIRY. YAY OR ...,2022-03-30 21:31:43,otown15,sci/tech,NEGATIVE
...,...,...,...,...,...,...
96,stocks,Help finding degrees for jobs related to finan...,2022-03-31 10:55:33,Bigdaddydamdam,business,POSITIVE
97,stocks,Focus group for Hedge Fund democratization,2022-03-31 10:55:59,spoiledfruit,business,POSITIVE
98,investing,Focus group for Stock and Crypto traders,2022-03-31 10:57:26,spoiledfruit,business,POSITIVE
99,pennystocks,"$LEAS, ANEW medical. Conference call presentat...",2022-03-31 10:58:25,DreamofMirrorz,business,NEGATIVE


In [33]:
lengths = []
for title in list_of_titles:
    lengths.append(len(title))

In [35]:
sum(lengths)

5392

In [1]:
import torch

if torch.cuda.is_available():
  print('GPU available')
else:
  print('Please set GPU via Edit -> Notebook Settings.')

GPU available


In [2]:
from simpletransformers.conv_ai import ConvAIModel

train_args = {
    "overwrite_output_dir": True,
    "reprocess_input_data": True,
    "use_early_stopping": True
}

model = ConvAIModel("gpt", "gpt_personachat_cache", use_cuda=True, args=train_args)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [40]:
personality = [
    "i like stocks",
    "i am from wallstreetbets",
    "i like reddit",
    "i love holding the stock"
]

response, history = model.interact_single(message="hello how are you?", history=[], personality=personality)
print(response)
response, history = model.interact_single(message="good! how much money do you make?", history=history, personality=personality)
print(response)
response, history = model.interact_single(message="what stock should i buy?", history=history, personality=personality)
print(response)

hello i am well. how are you?
i'm just a little over 70. i am just talking about funds.
stocks... not too much... just a little bit of insurance.
